In [4]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from tqdm.notebook import tqdm
from sknetwork.ranking import PageRank
from sknetwork.data import convert_edge_list, load_edge_list

In [27]:
network_files = {
    "BioPlex3": "processed_data/networks/BioPlex3_shared/edges_list_ncbi.csv",
    "HumanNet": "processed_data/networks/HumanNetV3/edges_list_ncbi.csv",
    "PCNet": "processed_data/networks/PCNet/edges_list_ncbi.csv",
    "ProteomeHD": "processed_data/networks/ProteomeHD/edges_list_ncbi.csv",
    "STRING": "processed_data/networks/STRING/edges_list_ncbi.csv",
}
networks = sorted(list(network_files.keys()))
alpha_array = [0.1, 0.2, 0.3, 0.5, 0.7, 0.9]
diseases=["asthma", "autism", "schizophrenia"]
networks

['BioPlex3', 'HumanNet', 'PCNet', 'ProteomeHD', 'STRING']

# Create Supra-adjacency matrix

In [14]:
# adj_matrices = []
# subgraphs_nodes = []


# for net in networks:
#     print("Loading {} graph".format(net))
#     df = pd.read_csv(network_files[net])[["node1", "node2"]].astype(str)
#     graph_nodes = set(df["node1"]).union(set(df["node2"]))
#     edge_list = list(df.itertuples(index=False))
#     graph = convert_edge_list(edge_list)
# #     node2idx = {n: i for i, n in enumerate(graph.names)}
# #     idx2node = {v: k for k, v in node2idx.items()}
#     adjacency = graph.adjacency
#     adj_matrices.append(adjacency)
#     subgraphs_nodes.append(graph.names)

In [16]:
gene_nets = {}

smat_edges = pd.DataFrame()
for net in networks:
    print("Loading {} graph".format(net))
    df = pd.read_csv(network_files[net])[["node1", "node2"]].astype(str)    
    graph_nodes = set(df["node1"]).union(set(df["node2"]))
    for nd in graph_nodes:
        if nd in gene_nets.keys():
            gene_nets[nd].append(net)
        else:
            gene_nets[nd] = [net]
            
    df["node1"] = net+"_"+df["node1"]
    df["node2"] = net+"_"+df["node2"]
    smat_edges = pd.concat((smat_edges, df))
smat_edges

Loading BioPlex3 graph
Loading HumanNet graph
Loading PCNet graph


/home/gvisona/anaconda3/envs/ml/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Loading ProteomeHD graph
Loading STRING graph


,node1,node2
0,BioPlex3_9782,BioPlex3_9991
1,BioPlex3_92014,BioPlex3_939
2,BioPlex3_91452,BioPlex3_9218
3,BioPlex3_91452,BioPlex3_9217
4,BioPlex3_9060,BioPlex3_9061
...,...,...
420529,STRING_381,STRING_100302736
420530,STRING_381,STRING_55605
420531,STRING_381,STRING_162
420532,STRING_381,STRING_23303


In [25]:
# Find inter-network edges
in_edges = []
tot_multigraph_genes = 0
for gn, nn in gene_nets.items():
    if len(nn)<2:
        continue
    tot_multigraph_genes += 1
    for i in range(0, len(nn)-1):
        for j in range(i+1, len(nn)):
#             print(nn[i], nn[j])
            in_edges.append((nn[i]+"_"+gn, nn[j]+"_"+gn))
in_edges = pd.DataFrame(in_edges, columns=["node1", "node2"])
print(tot_multigraph_genes)
in_edges

18692


,node1,node2
0,BioPlex3_85415,HumanNet_85415
1,BioPlex3_85415,PCNet_85415
2,BioPlex3_85415,STRING_85415
3,HumanNet_85415,PCNet_85415
4,HumanNet_85415,STRING_85415
...,...,...
82959,ProteomeHD_uniprot_Q5T3I0,STRING_uniprot_Q5T3I0
82960,ProteomeHD_4514,STRING_4514
82961,ProteomeHD_10011,STRING_10011
82962,ProteomeHD_uniprot_A8CG34,STRING_uniprot_A8CG34


In [23]:
# Complete Supra-matrix
smat_edges = pd.concat((smat_edges, in_edges))
smat_edges

,node1,node2
0,BioPlex3_9782,BioPlex3_9991
1,BioPlex3_92014,BioPlex3_939
2,BioPlex3_91452,BioPlex3_9218
3,BioPlex3_91452,BioPlex3_9217
4,BioPlex3_9060,BioPlex3_9061
...,...,...
82959,ProteomeHD_uniprot_Q5T3I0,STRING_uniprot_Q5T3I0
82960,ProteomeHD_4514,STRING_4514
82961,ProteomeHD_10011,STRING_10011
82962,ProteomeHD_uniprot_A8CG34,STRING_uniprot_A8CG34


In [26]:
edge_list = list(smat_edges.itertuples(index=False))
graph = convert_edge_list(edge_list)
node2idx = {n: i for i, n in enumerate(graph.names)}
idx2node = {v: k for k, v in node2idx.items()}
adjacency = graph.adjacency


In [38]:
for disease in diseases:
    print("\n\nAnalysing ", disease)
    
    data = pd.read_csv("processed_data/gwas_gene_pvals/{}/filtered_ncbi_PEGASUS_{}_gwas_data.csv".format(disease, disease))
    data = data[~data["NCBI_id"].isna()]
    data["NCBI_id"] = data["NCBI_id"].astype(str)
    min_pval = data["Pvalue"][data["Pvalue"]>0].min()
    pegasus_scores = {}
    for i, row in data.iterrows():
        pv = np.maximum(min_pval, np.minimum(1, row["Pvalue"]))
        pegasus_scores[row["NCBI_id"]] = np.maximum(0, -np.log10(pv))
    pegasus_ncbi_genes = set(pegasus_scores.keys())
    ncbi2gene = dict(zip(data.NCBI_id, data.Gene))
    # data[data["NCBI_id"].isin(graph["names"])].sort_values(by="Pvalue")
    
    
    pagerank_seeds = {}
    for node in graph["names"]:
        g_id = node.split("_", 1)[1]
        if g_id in pegasus_ncbi_genes:
            pagerank_seeds[node2idx[node]] = pegasus_scores[g_id]
        else:
            pagerank_seeds[node2idx[node]] = 0

    seeds_vector = np.array(list(pagerank_seeds.values()))
    nonzero_genes = [idx2node[g] for g in pagerank_seeds.keys() if pagerank_seeds[g]>0]
    max_val = np.max(seeds_vector[~np.isinf(seeds_vector)])
    
    
    def process_rwr_results(scores):
        rwr_results = []
        for i, node in enumerate(graph["names"]):
            row = {}
            g_id = node.split("_", 1)[1]

            row["Idx"] = node2idx[node]
            row["Network and NCBI ID"] = node
            row["Gene NCBI ID"] = g_id
            row["Symbol"] = ncbi2gene[g_id] if g_id in ncbi2gene.keys() else "-"


        #     row[""]
            if node in nonzero_genes:
                init_score = pagerank_seeds[node2idx[node]]
                if np.isinf(init_score):
                    init_score = max_val+1
            else:
                init_score = 0

            row["Initial Score"] = init_score
            row["Final Score"] = scores[i]

            rwr_results.append(row)

        rwr_results = pd.DataFrame(rwr_results).sort_values(by="Final Score", ascending=False)
        return rwr_results
    
    for alpha in tqdm(alpha_array):
        print("Alpha: ", alpha)
        pagerank = PageRank(damping_factor=alpha)
        rwr_scores = pagerank.fit_transform(adjacency, pagerank_seeds)
        rwr_results = process_rwr_results(rwr_scores)
        rwr_results = rwr_results.drop_duplicates(subset=["Gene NCBI ID"], keep="first") # Keep only the highest score across layers for each gene
        
        rwr_results.to_csv("outputs/MultiplexRWRs_scores/{}_multiplex_alpha{}_results.csv".format(disease, alpha), index=False)
        





Analysing  asthma


  0%|          | 0/6 [00:00<?, ?it/s]

Alpha:  0.1
Alpha:  0.2
Alpha:  0.3
Alpha:  0.5
Alpha:  0.7
Alpha:  0.9


Analysing  autism


  0%|          | 0/6 [00:00<?, ?it/s]

Alpha:  0.1
Alpha:  0.2
Alpha:  0.3
Alpha:  0.5
Alpha:  0.7
Alpha:  0.9


Analysing  schizophrenia


  0%|          | 0/6 [00:00<?, ?it/s]

Alpha:  0.1
Alpha:  0.2
Alpha:  0.3
Alpha:  0.5
Alpha:  0.7
Alpha:  0.9


In [39]:
rwr_results

,Idx,Network and NCBI ID,Gene NCBI ID,Symbol,Initial Score,Final Score
41647,41647,PCNet_7316,7316,UBC,0.113741,0.000698
61680,61680,STRING_7311,7311,UBA52,0.291932,0.000286
60273,60273,STRING_6233,6233,RPS27A,0.909569,0.000273
61681,61681,STRING_7314,7314,UBB,0.960751,0.000242
34893,34893,PCNet_351,351,APP,1.010904,0.000235
...,...,...,...,...,...,...
48158,48158,ProteomeHD_uniprot_P16278-3,uniprot_P16278-3,-,0.000000,0.000000
48474,48474,ProteomeHD_uniprot_Q9Y6K9-2,uniprot_Q9Y6K9-2,-,0.000000,0.000000
48134,48134,ProteomeHD_uniprot_O95819-3,uniprot_O95819-3,-,0.000000,0.000000
48446,48446,ProteomeHD_uniprot_Q9UKE5-2,uniprot_Q9UKE5-2,-,0.000000,0.000000


In [41]:
# for disease in diseases:
#     for alpha in alpha_array:
#         df = pd.read_csv("outputs/MultiplexRWRs_scores/{}_multiplex_alpha{}_results.csv".format(disease, alpha))
#         break
#     break

In [43]:
rwr_results

,Idx,Network and NCBI ID,Gene NCBI ID,Symbol,Initial Score,Final Score
41647,41647,PCNet_7316,7316,UBC,0.113741,0.000698
61680,61680,STRING_7311,7311,UBA52,0.291932,0.000286
60273,60273,STRING_6233,6233,RPS27A,0.909569,0.000273
61681,61681,STRING_7314,7314,UBB,0.960751,0.000242
34893,34893,PCNet_351,351,APP,1.010904,0.000235
...,...,...,...,...,...,...
48158,48158,ProteomeHD_uniprot_P16278-3,uniprot_P16278-3,-,0.000000,0.000000
48474,48474,ProteomeHD_uniprot_Q9Y6K9-2,uniprot_Q9Y6K9-2,-,0.000000,0.000000
48134,48134,ProteomeHD_uniprot_O95819-3,uniprot_O95819-3,-,0.000000,0.000000
48446,48446,ProteomeHD_uniprot_Q9UKE5-2,uniprot_Q9UKE5-2,-,0.000000,0.000000
